In [16]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split

from torchvision import transforms as T
from torchvision.datasets import ImageFolder
from torchvision.models.resnet import resnet18, ResNet18_Weights

import pytorch_lightning as pl

In [11]:
BS = 2

In [5]:
transform = T.Compose([
    T.Resize(126, T.InterpolationMode.NEAREST_EXACT),
    T.ToTensor()
])

In [12]:
dataset = ImageFolder('dataset', transform=transform)
train_set, test_set = random_split(dataset, [0.8, 0.2])

train_loader = DataLoader(train_set, BS, shuffle=True)
train_loader = DataLoader(test_set, BS, shuffle=False)

In [14]:
class Classifier(pl.LightningModule):
    def __init__(self, backbone, bb_out_dim):
        super().__init__()
        self.backbone = backbone
        self.head = nn.Linear(bb_out_dim, 10)

    def forward(self, x):
        x = self.backbone(x)
        x = self.head(x)

        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        loss = F.cross_entropy(self(x), y)
        self.log("train/loss", loss, prog_bar=True)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        loss = F.cross_entropy(self(x), y)
        self.log("val/loss", loss, prog_bar=True)

        return loss

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=1e-4)

In [19]:
backbone = resnet18(ResNet18_Weights.DEFAULT)
backbone.fc = nn.Identity()
model = Classifier(backbone, 512)
trainer = pl.Trainer(
    accelerator="auto",
    devices=1,
    max_epochs=1_000,
)

/home/yasin/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [20]:
trainer.fit(model, train_loader)

/home/yasin/anaconda3/lib/python3.11/site-packages/pytorch_lightning/trainer/configuration_validator.py:71: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name     | Type   | Params
------------------------------------
0 | backbone | ResNet | 11.2 M
1 | head     | Linear | 5.1 K 
------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.727    Total estimated model params size (MB)
/home/yasin/anaconda3/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

/home/yasin/anaconda3/lib/python3.11/site-packages/pytorch_lightning/trainer/call.py:53: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
